In [15]:
#!kill 2128689
!nvidia-smi

927.46s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Wed Oct  1 02:50:53 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 570.172.08             Driver Version: 570.172.08     CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4090        Off |   00000000:01:00.0 Off |                  Off |
|  0%   48C    P8             28W /  450W |    8310MiB /  24564MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [16]:
import sys
import os
import shutil
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
sys.path.append(os.path.abspath("../.."))
from loader.Dataset_image_based import VideoDataset_ImageBased
from torch.utils.data import DataLoader
from models.transformers.encoders.image_level.vit_like_encoder import ViT_ImageBased
from models.transformers.decoders.object_level.vit_decoder import ViT_Decoder
from models.transformers.CustomTransformer import CustomizableTransformer
from utils.util import count_model_params, train_epoch_image_based, eval_model_image_based, load_model, train_model
from matplotlib import pyplot as plt
from matplotlib import patches 
import torch
from torch.utils.tensorboard import SummaryWriter
from utils.loss_function import L1_SSIM_LPIPS_Loss5D_MemoryEfficient#ReconstructionLoss_L1_Ssim, ReconstructionLoss_PSNR_SSIM
from loader.transforms import RGBImageBasedNormalizer,Composition,CustomResize,RandomHorizontalFlip,RandomVerticalFlip,CustomColorJitter

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


#### configs

In [17]:

general_configs={
"data_path":"/home/nfs/inf6/data/datasets/MOVi/movi_c/",
"original_number_of_frames_per_video":24,
"selected_number_of_frames_per_video":4,
"batch_size":60,
"device" : torch.device("cuda" if torch.cuda.is_available() else "cpu"),
"img_height":128,
"patch_size":32,
"img_width":128,
"channels":3,
"learning_rate":3e-4,
"num_epochs":60,
"trainingMode":1 #0 training by bounding boxes, 1 training by masks. 
}

encoder_configs={
        "token_dim":128,
        "attn_dim":128,
        "num_heads":4,
        "mlp_size":512,
        "num_tf_layers":4
}
decoder_configs={
        "token_dim":128,
        "attn_dim":128,
        "num_heads":4,
        "mlp_size":512,
        "num_tf_layers":4
}

data_transform_config={
        "img_height":general_configs["img_height"],
        "img_width":general_configs["img_width"],
        "vFlip_probability":0.3,
        "hFlip_probability":0.3,
        "color_jitter_brightness":(0.8, 1.2),
        "color_jitter_hue":(-0.3, 0.3),
        "color_jitter_contrast":(0.6, 1.8),
        "color_jitter_saturation":(0.5, 1.5)
}

#### Datasets

In [18]:
transform_composition = Composition([
                                        RGBImageBasedNormalizer(),
                                        CustomResize((data_transform_config["img_height"],data_transform_config["img_width"])),
                                        #CustomColorJitter(
                                        #    brightness=data_transform_config["color_jitter_brightness"],
                                        #    hue=data_transform_config["color_jitter_hue"],
                                        #    contrast=data_transform_config["color_jitter_contrast"],
                                        #    saturation=data_transform_config["color_jitter_saturation"]
                                        #)
                                    ])
validation_dataset = VideoDataset_ImageBased(data_path=general_configs["data_path"],
                            split='validation',
                            original_number_of_frames_per_video=general_configs["original_number_of_frames_per_video"],
                            selected_number_of_frames_per_video=general_configs["selected_number_of_frames_per_video"],
                            halve_dataset=True,
                            is_test_dataset=False,
                            transforms=transform_composition)
valid_loader = DataLoader(dataset=validation_dataset,
                            batch_size=general_configs["batch_size"],
                            shuffle=False,
                            drop_last=False)
train_dataset = VideoDataset_ImageBased(data_path=general_configs["data_path"],
                            split='train',
                            original_number_of_frames_per_video=general_configs["original_number_of_frames_per_video"],
                            selected_number_of_frames_per_video=general_configs["selected_number_of_frames_per_video"],
                            halve_dataset=False,
                            is_test_dataset=False,
                            transforms=transform_composition)
train_loader = DataLoader(dataset=train_dataset,
                            batch_size=general_configs["batch_size"],
                            shuffle=True,
                            drop_last=True)

#### Datashapes

In [19]:
iterator=iter(train_loader)
rgbs=next(iterator)
print(f"shapes: {rgbs.shape=},\r\n============================================")
print(len(train_loader))

shapes: rgbs.shape=torch.Size([60, 4, 3, 128, 128]),
162


### Encoder

In [20]:
def defineVIT():
        return  ViT_ImageBased(
                patch_size=general_configs["patch_size"],
                img_size=rgbs.shape[-2],
                frame_numbers=general_configs["selected_number_of_frames_per_video"],
                token_dim=encoder_configs["token_dim"],
                attn_dim=encoder_configs["attn_dim"],
                num_heads=encoder_configs["num_heads"],
                mlp_size=encoder_configs["mlp_size"],
                num_tf_layers=encoder_configs["num_tf_layers"]).to(general_configs["device"])

In [21]:
vit=defineVIT()
print(f"ViT has {count_model_params(vit)} parameters")
vit

ViT has 1192704 parameters


ViT_ImageBased(
  (patch_projection): Sequential(
    (0): LayerNorm((3072,), eps=1e-05, elementwise_affine=True)
    (1): Linear(in_features=3072, out_features=128, bias=True)
  )
  (pos_emb): PositionalEncoding()
  (transformer_blocks): Sequential(
    (0): EncoderBlock(
      (ln_att): LayerNorm((128,), eps=1e-06, elementwise_affine=True)
      (attn): MultiHeadAttention(
        (q): Linear(in_features=128, out_features=128, bias=True)
        (k): Linear(in_features=128, out_features=128, bias=True)
        (v): Linear(in_features=128, out_features=128, bias=True)
        (out_proj): Linear(in_features=128, out_features=128, bias=True)
      )
      (ln_mlp): LayerNorm((128,), eps=1e-06, elementwise_affine=True)
      (mlp): MLP(
        (mlp): Sequential(
          (0): Linear(in_features=128, out_features=512, bias=True)
          (1): GELU(approximate='none')
          (2): Linear(in_features=512, out_features=128, bias=True)
        )
      )
    )
    (1): EncoderBlock(
     

### Decoder

In [22]:
def defineDecoder():
    return ViT_Decoder(
    batch_size=general_configs["batch_size"],
    img_height=general_configs["img_height"],
    img_width=general_configs["img_width"],
    channels=general_configs["channels"],
    frame_numbers=general_configs["selected_number_of_frames_per_video"],
    token_dim=decoder_configs["token_dim"],
    attn_dim=decoder_configs["attn_dim"], 
    num_heads=decoder_configs["num_heads"], 
    mlp_size=decoder_configs["mlp_size"], 
    num_tf_layers=decoder_configs["num_tf_layers"],
    device=general_configs["device"]
).to(general_configs["device"])

In [23]:
decoder=defineDecoder()
print(f"Decoder has {count_model_params(decoder)} parameters")
decoder

Decoder has 1246115 parameters


ViT_Decoder(
  (patch_projection): Sequential(
    (0): ConvTranspose2d(128, 256, kernel_size=(4, 4), stride=(1, 1))
    (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (7): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU(inplace=True)
    (9): ConvTranspose2d(64, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (10): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): ReLU(inplace=True)
    (12): ConvTranspose2d(32, 3, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (13): Sigmoid()
  )
  (pos_emb): PositionalEncoding()
)

### Transformer

In [24]:
transformer=CustomizableTransformer(encoder=vit, decoder=decoder).to(general_configs["device"])
assert count_model_params(decoder)+count_model_params(vit)==count_model_params(transformer)
print(f"transformer has {count_model_params(transformer)} parameters")
transformer

transformer has 2438819 parameters


CustomizableTransformer(
  (encoder): ViT_ImageBased(
    (patch_projection): Sequential(
      (0): LayerNorm((3072,), eps=1e-05, elementwise_affine=True)
      (1): Linear(in_features=3072, out_features=128, bias=True)
    )
    (pos_emb): PositionalEncoding()
    (transformer_blocks): Sequential(
      (0): EncoderBlock(
        (ln_att): LayerNorm((128,), eps=1e-06, elementwise_affine=True)
        (attn): MultiHeadAttention(
          (q): Linear(in_features=128, out_features=128, bias=True)
          (k): Linear(in_features=128, out_features=128, bias=True)
          (v): Linear(in_features=128, out_features=128, bias=True)
          (out_proj): Linear(in_features=128, out_features=128, bias=True)
        )
        (ln_mlp): LayerNorm((128,), eps=1e-06, elementwise_affine=True)
        (mlp): MLP(
          (mlp): Sequential(
            (0): Linear(in_features=128, out_features=512, bias=True)
            (1): GELU(approximate='none')
            (2): Linear(in_features=512, out

#### Training

In [25]:
criterion=L1_SSIM_LPIPS_Loss5D_MemoryEfficient(l1_lambda=1.0,ssim_lambda=0.5, lpips_lambda=0.1)
optimizer = torch.optim.Adam(transformer.parameters(), lr=general_configs["learning_rate"])
scheduler = torch.optim.lr_scheduler.MultiplicativeLR(optimizer, lr_lambda=lambda epoch: 0.95)
TBOARD_LOGS = os.path.join(os.getcwd(), "../tboard_logs", "ViT_30")
if not os.path.exists(TBOARD_LOGS):
    os.makedirs(TBOARD_LOGS)

shutil.rmtree(TBOARD_LOGS)
writer = SummaryWriter(TBOARD_LOGS)

Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]


/home/user/masroora1/VideoTracker/.venv/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/user/masroora1/VideoTracker/.venv/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Loading model from: /home/user/masroora1/VideoTracker/.venv/lib/python3.12/site-packages/lpips/weights/v0.1/alex.pth


In [26]:
train_loss, val_loss, loss_iters, valid_acc = train_model(
        model=transformer,
        optimizer=optimizer,
        scheduler=scheduler,
        criterion=criterion,
        train_loader=train_loader,
        valid_loader=valid_loader,
        num_epochs=general_configs["num_epochs"],
        tboard=writer,
        trainingmode=general_configs["trainingMode"],
        saveImagesPerEachEpoch=True,
        eval_model_func=eval_model_image_based,
        train_epoch_func=train_epoch_image_based
           )

Started Epoch 1/60...
  --> Running valdiation epoch


RuntimeError: shape '[60, 4, 3, 128, 128]' is invalid for input of size 2949120

In [ ]:
from utils.util import save_model
stats = {
    "train_loss": train_loss,
    "valid_loss": val_loss,
    "loss_iters": loss_iters,
    "valid_acc": valid_acc
}
save_model(transformer, optimizer, epoch=general_configs["num_epochs"], stats=stats)

In [ ]:
"""
stats = {
    "train_loss": [],
    "valid_loss": [],
    "loss_iters": [],
    "valid_acc": []
}
model, optimizer, epoch, stats = load_model(transformer, optimizer, savepath="../../checkpoints/checkpoint_epoch_60_SSIM_L1.pth")"""

In [ ]:
from utils.util import visualize_progress

loss_iters = stats['loss_iters']
val_loss = stats['valid_loss']
train_loss = stats['train_loss']
valid_acc = stats['valid_acc']

visualize_progress(loss_iters, train_loss, val_loss, valid_acc, start=0)
plt.show()

In [ ]:
test_dataset = VideoDataset_ImageBased(data_path=general_configs["data_path"],
                            split='validation',
                            original_number_of_frames_per_video=general_configs["original_number_of_frames_per_video"],
                            selected_number_of_frames_per_video=general_configs["selected_number_of_frames_per_video"],
                            max_objects_in_scene=general_configs["max_objects_in_scene"],
                            halve_dataset=True,
                            is_test_dataset=True,
                            transforms=transform_composition)
test_loader = DataLoader(dataset=test_dataset,
                            batch_size=general_configs["batch_size"],
                            shuffle=False,
                            drop_last=True)

In [ ]:
import random


iterator=iter(test_loader)
coms,bboxes,masks,rgbs,flows=next(iterator)
transformer.eval()
with torch.no_grad():
   recons = transformer(rgbs.to(device),masks=masks.to(device)).to('cpu')

number_of_images=10
number_of_columns=2
video_index=random.randint(0,recons.shape[0])
fig, ax = plt.subplots(number_of_images,number_of_columns)
fig.set_size_inches(number_of_columns*5, number_of_images*3)

for i in range (number_of_images):
    for i in range(number_of_images):
        ax[i,1].imshow((recons[video_index,i]*255).clamp(0, 255).permute(1,2,0).byte().numpy())
        ax[i,1].axis("off")
        ax[i,0].imshow((rgbs[video_index,i]*255).clamp(0, 255).permute(1,2,0).byte().numpy())
        ax[i,0].axis("off")
        if i==0:
            ax[i,1].set_title(f"Reconstructed Image {i+1}")    
            ax[i,0].set_title(f"Original Image {i+1}")    


plt.tight_layout()
plt.show()